# Mental Health in Tech: Predicting Treatment-Seeking Behavior

## Second Notebook : Objective

This notebook prepares the Mental Health in Tech Survey dataset for supervised machine learning.
After the initial exploratory data analysis (EDA), this step focuses on:

   -Cleaning inconsistent and missing values
        
   -Normalizing categories (e.g., Gender)
        
   -Handling outliers (e.g., Age)
        
   -Encoding categorical and ordinal variables
        
   -Preparing the final dataset for modeling

The cleaned and encoded dataset will serve as input for the classification models developed in the next notebook.

**Author:** J-F Jutras  
**Date:** July-August 2025  
**Dataset:** [Mental Health in Tech Survey (Kaggle)](https://www.kaggle.com/datasets) 


## 2.1-Data Loading

The dataset is retrieved directly from Kaggle using the `kagglehub` library. This ensures the analysis remains reproducible and uses the latest available version.

In [1]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("osmi/mental-health-in-tech-survey")

#Load dataset directly from Kaggle cache
df = pd.read_csv(f"{path}/survey.csv")

#Quick check
df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


## 2.2-Data Cleaning and Feature Engineering

Before building predictive models, this section focuses on preparing the *Mental Health in Tech Survey* dataset for analysis. The following steps outline the systematic approach to cleaning and transforming the data for machine learning.

---

**1. Handling Missing and Inconsistent Data**
- Identify all columns with missing or inconsistent entries.  
- Apply appropriate strategies to handle missing values.  
- Verify data completeness after imputation or correction.  

---

**2. Cleaning and Normalizing Categorical Variables**
- Standardize inconsistent category labels (e.g., capitalization, spelling, synonyms).  
- Consolidate rare or ambiguous categories where relevant.  
- Ensure all categorical variables contain consistent, interpretable values.  

---

**3. Handling Outliers in Numerical Data**
- Review distributions of numerical variables.  
- Detect and handle potential outliers that could bias the analysis.  
- Validate cleaned values with visualizations.  

---

**4. Encoding Categorical/Ordinal Variables and Feature Engineering**
- Convert categorical data into numerical format suitable for modeling.  
- Apply appropriate encoding strategies depending on the variable type.  
- Verify resulting dataset structure and dimensions.  
- Evaluate whether additional features should be created or transformed to improve model interpretability and performance.  


In [2]:
#1-HANDLING MISSING AND INCONSISTANT DATA

#Count missing values
missing_values = df.isnull().sum().sort_values(ascending = False)
print(missing_values)

comments                     1095
state                         515
work_interfere                264
self_employed                  18
seek_help                       0
obs_consequence                 0
mental_vs_physical              0
phys_health_interview           0
mental_health_interview         0
supervisor                      0
coworkers                       0
phys_health_consequence         0
mental_health_consequence       0
leave                           0
anonymity                       0
Timestamp                       0
wellness_program                0
Age                             0
benefits                        0
tech_company                    0
remote_work                     0
no_employees                    0
treatment                       0
family_history                  0
Country                         0
Gender                          0
care_options                    0
dtype: int64


In [3]:
#Since many respondents are international (not from the US), they naturally left the 'state'
#field blank.To confirm that missing 'state' values mostly come from US respondents, 
#let's check how many Americans did not provide their state information.

# Filter only US respondents
us_respondents = df[df['Country'] == 'United States']

# Count missing state values among them
missing_us_states = us_respondents['state'].isnull().sum()
print(f"Missing 'state' entries among US respondents: {missing_us_states}")


Missing 'state' entries among US respondents: 11


In [4]:
#Replace missing state only for US respondents
#11 US respondents did not provide their state
df.loc[(df['Country'] == 'United States') & (df['state'].isnull()), 'state'] = 'Unknown'

#Verify
missing_us_states = df.loc[df['Country'] == 'United States', 'state'].isnull().sum()
print(f"Missing 'state' entries among US respondents after filling: {missing_us_states}")

Missing 'state' entries among US respondents after filling: 0


In [5]:
#Fill missing work_interfere values with "Unknown" (264 respondents didn't answer this question)
#We replace NaN with "Unknown" to:
#1) Preserve all observations for analysis and modeling.
#2) Allow later analysis to see if non-response ("Unknown") itself is correlated with the target variable.
#3) Keep context: The question asked was "If you have a mental health condition, do you feel that it interferes with your work?"
#Is it possible that these people didn't want to answer?
df['work_interfere'] = df['work_interfere'].fillna('Unknown')

# Verify
missing_work_interfere = df['work_interfere'].isnull().sum()
print(f"Missing 'work_interfere' entries after filling: {missing_work_interfere}")


Missing 'work_interfere' entries after filling: 0


In [6]:
#Drop rows where self_employed is missing since it's a very small proportion (~1.4%).
#The missing information doesn't seem to be significant or meaningful for analysis.
df = df.dropna(subset=['self_employed'])

# Verify
print(f"Remaining missing 'self_employed' entries: {df['self_employed'].isnull().sum()}", "\n")
print(df.isnull().sum().sort_values(ascending = False))

Remaining missing 'self_employed' entries: 0 

comments                     1080
state                         497
wellness_program                0
obs_consequence                 0
mental_vs_physical              0
phys_health_interview           0
mental_health_interview         0
supervisor                      0
coworkers                       0
phys_health_consequence         0
mental_health_consequence       0
leave                           0
anonymity                       0
seek_help                       0
Timestamp                       0
Age                             0
benefits                        0
tech_company                    0
remote_work                     0
no_employees                    0
work_interfere                  0
treatment                       0
family_history                  0
self_employed                   0
Country                         0
Gender                          0
care_options                    0
dtype: int64


In [7]:
#2-CLEANING AND NORMALIZING CATEGORICAL VARIABLES

#This Gender categorization is aligned with the HR DEI definitions currently in use:
#-'Male' and 'Female' represent cisgender identities clearly identifying as male or female.
#-'LGBTQ2+' includes trans, non-binary, genderqueer, fluid, and other diverse gender identities.
#-'Not Specified' captures respondents who skip the question or provide ambiguous/non-meaningful answers.
#These categories can be reviewed and clarified as needed to ensure inclusivity and accuracy.

#Standardize strings
df['Gender_clean'] = df['Gender'].str.strip().str.lower()

#Define category lists
male_terms = ['m','male','mal','male-ish','male (cis)','cis male','man','mail','msle','make','malr','male leaning androgynous','guy (-ish) ^_^','ostensibly male, unsure what that really means']
female_terms = ['f','female','femake','woman','cis female','woman','female ']
lgbtq_terms = ['trans-female','trans woman','female (trans)','non-binary','queer/she/they','enby','fluid','genderqueer','androgyne','agender','something kinda male?','something kinda female?']
not_specified_terms = ['nah','all','a little about you','p']

#Function to categorize
def categorize_gender(x):
    if x in male_terms:
        return 'Male'
    elif x in female_terms:
        return 'Female'
    elif x in lgbtq_terms:
        return 'LGBTQ2+'
    else:
        return 'Not Specified'

df['Gender_clean'] = df['Gender_clean'].apply(categorize_gender)

#Check distribution
print(df['Gender_clean'].value_counts())

Gender_clean
Male             979
Female           239
LGBTQ2+           12
Not Specified     11
Name: count, dtype: int64


In [8]:
#3-HANDLING OUTLIERS IN NUMERICAL DATA

#Check Age Distribution
print(df['Age'].value_counts().sort_index())

Age
-1726            1
-29              1
-1               1
 5               1
 8               1
 11              1
 18              7
 19              9
 20              6
 21             16
 22             21
 23             49
 24             46
 25             61
 26             75
 27             70
 28             68
 29             83
 30             63
 31             64
 32             80
 33             69
 34             65
 35             54
 36             36
 37             42
 38             39
 39             32
 40             33
 41             21
 42             18
 43             28
 44             10
 45             12
 46             12
 47              2
 48              6
 49              4
 50              6
 51              5
 53              1
 54              3
 55              3
 56              4
 57              3
 58              1
 60              2
 61              1
 62              1
 65              1
 72              1
 329             1
 9999999

In [9]:
#The Age column contains implausible values due to data entry errors (e.g., negative ages, extremely low or high numbers).
#These outliers can bias modeling, especially for supervised ML algorithms.
#We drop all rows where Age is outside the plausible human range of 18–72 years.

df = df[(df['Age'] >= 18) & (df['Age'] <= 72)]

#Verify remaining distribution
print(df['Age'].value_counts().sort_index())

Age
18     7
19     9
20     6
21    16
22    21
23    49
24    46
25    61
26    75
27    70
28    68
29    83
30    63
31    64
32    80
33    69
34    65
35    54
36    36
37    42
38    39
39    32
40    33
41    21
42    18
43    28
44    10
45    12
46    12
47     2
48     6
49     4
50     6
51     5
53     1
54     3
55     3
56     4
57     3
58     1
60     2
61     1
62     1
65     1
72     1
Name: count, dtype: int64


In [10]:
#4-ENCODING CATEGORICAL/ORDINAL VARIABLES AND FEATURE ENGINEERING
#To reduce sparsity after encoding, let's group rare countries into an 'Other' category

#Calculate the proportion of each country
country_freq = df['Country'].value_counts(normalize=True)

#Define a threshold below which a country is considered rare
threshold = 0.02  # 2% of respondents

#Identify rare countries
rare_countries = country_freq[country_freq < threshold].index.tolist()

#Replace rare countries with 'Other'
df['Country_clean'] = df['Country'].apply(lambda x: 'Other' if x in rare_countries else x)

#Check the new distribution
print(df['Country_clean'].value_counts())

Country_clean
United States     735
United Kingdom    182
Other             149
Canada             68
Germany            45
Netherlands        27
Ireland            27
Name: count, dtype: int64


In [11]:
#Encode ordinal variable: work_interfere
#Ordered encoding is appropriate here because the responses have a clear progression:
#"Never" < "Rarely" < "Sometimes" < "Often" < "Unknown".
work_order = ['Never', 'Rarely', 'Sometimes', 'Often', 'Unknown']
df['work_interfere_encoded'] = pd.Categorical(
    df['work_interfere'],
    categories = work_order,
    ordered = True
).codes

#One-hot encode 'no_employees' instead of using ordinal encoding.
df = pd.get_dummies(df, columns = ['no_employees'], prefix = 'no_emp', drop_first = True)

#One-hot encode remaining nominal categorical variables
nominal_vars = [
    'Gender_clean', 'Country_clean', 'state', 'self_employed', 'family_history',
    'tech_company', 'benefits', 'care_options', 'wellness_program',
    'seek_help', 'anonymity', 'leave', 'mental_health_consequence',
    'phys_health_consequence', 'coworkers', 'supervisor',
    'mental_health_interview', 'phys_health_interview',
    'mental_vs_physical', 'obs_consequence', 'remote_work'
]

df = pd.get_dummies(df, columns = nominal_vars, drop_first = True)

#Verify dataset
print(f"Dataset shape after encoding: {df.shape}")
df.head()

Dataset shape after encoding: (1233, 100)


,Timestamp,Age,Gender,Country,treatment,work_interfere,comments,work_interfere_encoded,no_emp_100-500,no_emp_26-100,...,supervisor_Some of them,supervisor_Yes,mental_health_interview_No,mental_health_interview_Yes,phys_health_interview_No,phys_health_interview_Yes,mental_vs_physical_No,mental_vs_physical_Yes,obs_consequence_Yes,remote_work_Yes
18,2014-08-27 11:34:53,46,male,United States,No,Sometimes,NaN,2,False,False,...,False,True,True,False,False,True,False,True,True,True
19,2014-08-27 11:35:08,36,Male,France,No,Unknown,NaN,4,False,False,...,True,False,False,False,False,False,False,False,False,True
20,2014-08-27 11:35:12,29,Male,United States,Yes,Sometimes,NaN,2,True,False,...,True,False,True,False,True,False,True,False,False,False
21,2014-08-27 11:35:24,31,male,United States,No,Never,NaN,0,False,False,...,True,False,True,False,False,False,False,True,False,True
22,2014-08-27 11:35:48,46,Male,United States,Yes,Often,NaN,3,False,True,...,False,True,True,False,False,False,True,False,False,True


In [12]:
#Encode target variable: 'No' → 0, 'Yes' → 1
df['treatment_encoded'] = df['treatment'].map({'No': 0, 'Yes': 1})

#Verify
print(df['treatment_encoded'].value_counts())

treatment_encoded
1    623
0    610
Name: count, dtype: int64


In [13]:
#The Age variable is continuous but not linearly related to mental health outcomes.
#Grouping ages into bins captures meaningful life or career stages (e.g., early career vs. mid-career).
#Because these bins represent discrete categories rather than a continuous progression,
#one-hot encoding is more appropriate than ordinal encoding.

#Define age bins and labels
age_bins = [17, 25, 35, 45, 55, 73]  # Note: 56-72 merged into 56+
age_labels = ['18-25', '26-35', '36-45', '46-55', '56+']

#Create the binned variable
df['Age_binned'] = pd.cut(df['Age'], bins = age_bins, labels = age_labels, right = True, include_lowest = True)

#Verify distribution before encoding
print("Age bins distribution before encoding:")
print(df['Age_binned'].value_counts().sort_index(), "\n")

#One-hot encode the new categorical variable
df = pd.get_dummies(df, columns = ['Age_binned'], prefix = 'Age', drop_first = True)

#Verify one-hot encoded columns
print("Sample of one-hot encoded Age columns:")
print(df.filter(like = 'Age_').head())

Age bins distribution before encoding:
Age_binned
18-25    215
26-35    691
36-45    271
46-55     42
56+       14
Name: count, dtype: int64 

Sample of one-hot encoded Age columns:
    Age_26-35  Age_36-45  Age_46-55  Age_56+
18      False      False       True    False
19      False       True      False    False
20       True      False      False    False
21       True      False      False    False
22      False      False       True    False


In [14]:
#Create a dataset ready for comment analysis
#Drop non-predictive and redundant columns
to_drop = [
    'Timestamp',       
    'treatment',                   # Original target
    'Gender', 'Country', 'Country_clean', 'state', 'no_employees',  # Replaced by encoded
    'work_interfere', 'Age'        # Replaced by encoded versions
]

df_comments = df.drop(columns=to_drop, errors='ignore')

#Convert boolean columns to numeric (1/0), but leave 'comments' as text
bool_cols = df_comments.select_dtypes(include = 'bool').columns
df_comments[bool_cols] = df_comments[bool_cols].astype(int)

#Replace missing values in 'comments' with empty string
df_comments['comments'] = df_comments['comments'].fillna('')

#Verify remaining non-numeric columns
print("Non-numeric columns remaining:", df_comments.select_dtypes(exclude=['number']).columns.tolist())
print(f"Dataset for comment analysis shape: {df_comments.shape}")

#Export the cleaned dataset for comment analysis
df_comments.to_csv('/kaggle/working/mental_health_comments.csv', index = False)
print("Dataset exported as 'mental_health_comments.csv'.")

Non-numeric columns remaining: ['comments']
Dataset for comment analysis shape: (1233, 99)
Dataset exported as 'mental_health_comments.csv'.


In [15]:
#Create a modeling-ready dataset
#Drop non-predictive and redundant columns
to_drop = [
    'Timestamp', 'comments',       #May be used later for exploratory or qualitative analysis
    'treatment',                   # Original target
    'Gender', 'Country', 'Country_clean', 'state', 'no_employees',  # Replaced by encoded
    'work_interfere', 'Age'        # Replaced by encoded versions
]

df_model = df.drop(columns=to_drop, errors='ignore')

#Convert all boolean columns (True/False) to numeric (1/0)
df_model = df_model.astype(int)

#Verify only numeric columns remain
print("Non-numeric columns remaining:", df_model.select_dtypes(exclude = ['number']).columns.tolist())
print(f"Final modeling dataset shape: {df_model.shape}")

Non-numeric columns remaining: []
Final modeling dataset shape: (1233, 98)


## 2.3–Summary

The dataset has been successfully cleaned, standardized, and encoded for supervised machine learning.

**Key preparation steps:**
- Missing and inconsistent data handled appropriately *(e.g., `state`, `work_interfere`, `self_employed`)*
- Gender categories normalized into four inclusive groups *(Male, Female, LGBTQ2+, Not Specified)*
- Implausible age values removed *(kept range: 18–72)*
- Rare countries grouped under **"Other"** to reduce sparsity
- Ordinal and nominal categorical variables encoded *(ordered encoding for `work_interfere`, one-hot encoding for others)*
- Age transformed into career-stage bins and one-hot encoded
- All features converted to numeric for modeling compatibility  

**Final dataset summary:**
- 1233 observations
- 98 fully numeric features
- No remaining missing or non-numeric values

**Columns dropped:**  
`Timestamp`, `comments`, `treatment`, `Gender`, `Country`, `Country_clean`,`state`, `no_employees`, `work_interfere`, `Age` removed for redundancy, irrelevance, or replacement by encoded versions.

The resulting dataset (**`df_model`**) is now ready for supervised classification modeling in the next notebook, where predictive algorithms will be trained to analyze **mental health treatment-seeking behavior** among tech professionals.


## 2.4–Save the Cleaned Dataset

In [16]:
#Define Kaggle output path
output_path = "/kaggle/working/mental_health_cleaned.csv"

#Save cleaned dataset
df_model.to_csv(output_path, index=False)

print("Cleaned dataset successfully saved!")
print(f"File location: {output_path}")
print(f"Shape: {df_model.shape[0]} rows × {df_model.shape[1]} columns")

#Preview first rows
df_model.head()

Cleaned dataset successfully saved!
File location: /kaggle/working/mental_health_cleaned.csv
Shape: 1233 rows × 98 columns


,work_interfere_encoded,no_emp_100-500,no_emp_26-100,no_emp_500-1000,no_emp_6-25,no_emp_More than 1000,Gender_clean_LGBTQ2+,Gender_clean_Male,Gender_clean_Not Specified,Country_clean_Germany,...,phys_health_interview_Yes,mental_vs_physical_No,mental_vs_physical_Yes,obs_consequence_Yes,remote_work_Yes,treatment_encoded,Age_26-35,Age_36-45,Age_46-55,Age_56+
18,2,0,0,0,0,0,0,1,0,0,...,1,0,1,1,1,0,0,0,1,0
19,4,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
20,2,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,1,0,0,0
21,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,1,0,0,0
22,3,0,1,0,0,0,0,1,0,0,...,0,1,0,0,1,1,0,0,1,0
